# 加载训练好的词嵌入数据
因为词嵌入的训练是非常耗资源的，所以大部分人都是选择加载训练好的词嵌入数据。在本博客中，我们将学习到：

- 如何加载训练好了的词向量
- 使用余弦相似性计算相似度
- 使用词嵌入来解决“男人与女人相比就像国王与____ 相比”之类的词语类比问题
- 修改词嵌入以减少性别偏见等
我们先来加载所需要的包：

In [1]:
import numpy as np
import w2v_utils

Using TensorFlow backend.


接下来就是加载词向量了，这里我们使用50维的向量来表示单词：

In [2]:
words, word_to_vec_map = w2v_utils.read_glove_vecs('data/glove.6B.50d.txt')

我们加载了以下数据：

- words：单词的集合
- word_to_vec_map ： 字典类型，单词到GloVe向量的映射

你可以使用下面的语句来看看到底保存了些什么：

因为独热向量不能很好地表示词语词之间的相似性，所以使用了GloVe向量，它保存了每个单词更多、更有用的信息，我们现在可以看看如何比较两个词的相似性。

# 余弦相似度
  为了衡量两个词的相似程度，我们需要一种方法来衡量两个词的词嵌入向量之间的相似程度，给定两个向量$u$和$v$，余弦相似度定义如下:
  $$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$
  $u$与$v$之间的相似度是基于他们之间的角度计算的，它们越相似，那么$cos(\theta)$的值就越接近于1；如果它们很不相似，那么他们的夹角就越大，$cos(\theta)$的值就越接近于-1。



In [5]:
def cosine_similarity(u, v):
    """
    u与v的余弦相似度反映了u与v的相似程度
    
    参数：
        u -- 维度为(n,)的词向量
        v -- 维度为(n,)的词向量
        
    返回：
        cosine_similarity -- 由上面公式定义的u和v之间的余弦相似度。
    """
    distance = 0
    
    # 计算u与v的内积
    dot = np.dot(u, v)
    
    #计算u的L2范数
    norm_u = np.sqrt(np.sum(np.power(u, 2)))
    
    #计算v的L2范数
    norm_v = np.sqrt(np.sum(np.power(v, 2)))
    
    # 根据公式1计算余弦相似度
    cosine_similarity = np.divide(dot, norm_u * norm_v)
    
    return cosine_similarity

In [6]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.27439246261379424
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174202


# 词类类比
在这里，我们将学习解决“A与B相比就类似于C与____相比一样”之类的问题，打个比方，“男人与女人相比就像国王与 女皇 相比”

In [ ]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    解决“A与B相比就类似于C与____相比一样”之类的问题
    
    参数：
        word_a -- 一个字符串类型的词
        word_b -- 一个字符串类型的词
        word_c -- 一个字符串类型的词
        word_to_vec_map -- 字典类型，单词到GloVe向量的映射
        
    返回：
        best_word -- 满足(v_b - v_a) 最接近 (v_best_word - v_c) 的词
    """
    
    # 把单词转换为小写
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    # 获取对应单词的词向量
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    # 获取全部的单词
    words = word_to_vec_map.keys()
    
    # 将max_cosine_sim初始化为一个比较大的负数
    max_cosine_sim = -100
    best_word = None
    
    # 遍历整个数据集
    for word in words:
        # 要避免匹配到输入的数据
        if word in [word_a, word_b, word_c]:
            continue
        # 计算余弦相似度
        cosine_sim = cosine_similarity((e_b - e_a), (word_to_vec_map[word] - e_c))
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = word
            
    return best_word